In [1]:
import shutil
import re 
from glob import glob
import os
from tqdm import tqdm

import librosa
import soundfile as sf
import numpy as np 
import random
import pandas as pd

from functional import *
from overlap import get_trimmed_sample, overlap_using_min

In [2]:
sr = 16000

In [3]:
paths = {"sample1": "C:/Users/b.smadiarov/Diploma/DualPathRNN/samples/5ed8a1c0f3ea2.flac",
         "sample2": "C:/Users/b.smadiarov/Diploma/DualPathRNN/samples/5f2b0a06ceb18.flac",
         
         'KS2': 'F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/',
         'root': 'F:/ISSAI_KSC2_unpacked',
         
         'src_csv': 'C:/Users/b.smadiarov/Diploma/VoiceDetectionForKazakhSpeech/data/annotation/Csv', 

         'trimmed': 'F:/ISSAI_KSC2_unpacked/trimmed', 
         "mixed": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/mixed", 
         "samples": "C:/Users/b.smadiarov/Diploma/DualPathRNN/samples",
         "tts": 'F:/ISSAI_KSC2_unpacked/trimmed/Train/tts'
        }

glob_paths = {'KS2_raw': 'F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/**/**/*.flac',
              "mixed_csv": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/mixed/*.csv",
              "mixed_flac": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/mixed/*.flac",

              "f_real_flac": 'F:/ISSAI_KSC2_unpacked/trimmed/**/**/*.flac',
              "tts": 'F:/ISSAI_KSC2_unpacked/trimmed/Train/tts/*.flac', 
             } 

#### Generate samples

ToDO: 
- Короткие записи сливались с короткими записями.

Критерий: нету посторонних звуков 
1. tts - только для text_to_speech
2. Самый лучший crowdsourced,
3. tv_news: эпортаж это обычно речь 
4. parlament: т.к. кричат с требун  
5. Ужасные - radio: т.к. играет вечно музыка, talkshow - тоже какая то дичь, podcast - казахские подкасты это не подкасты а радио.

Кроме памяти (мне ее жалко) и так же в аннотационных файлах указана путь до файла. Поэтому что бы придестеречься 
я решил оставить все как есть.

##### **1. Убираем начало и конец.**

In [ ]:
tracklist = get_audios_with_folder_name(['crowdsourced', 'tts', 'tv_news'], glob_paths['KS2_raw'])
print('Dataset total len:', len(tracklist))

In [5]:
for t in tqdm(tracklist):
    trimmed_audio = get_trimmed_sample(t, sr, paths['src_csv'])
    folder = t.replace("ISSAI_KSC2/", "trimmed/").replace(f"/{get_file_name (t)}.flac", '')
    os.makedirs(folder, exist_ok=True)
    path_to_save = folder + f"/{get_file_name (t)}.flac"
    sf.write(path_to_save, trimmed_audio, samplerate = sr)

  0%|          | 0/516458 [00:00<?, ?it/s]

##### **2. Объединяем аудиодорожки**

Pipline: 
1. Есть большой набор данных - trimmed в нем только самые лучшие для задачи диаризации аудодорожки. Сэмплов достаточно много - поэтому будет переменная `total_amount`, которая будет брать уже от trimmed папки.
2. Далее т.к. мы объединяем аудиодорожки то нам нужно их по отдельности обрезанное сохранить или запомнить их общую длину в csv файлы. В csv файле будет храниться столбец `min_length`который будет означать общую минимальную длину. Это поможет экономить место на диске. Но из за этого будет долше проходить инициализация.

In [4]:
total_amount_percent = 0.1 # 10 %
# Примерное число которое я хочу ~ 2000 записей для начало
total_amount = 2000
k = 2
seed = k

1. Число класстеров _k_
2. _Seed_ зависит от числа кластеров. Это сделанно для того что бы добавить не предусмотренность. Но таким образом есть опасность что модель просто выучит голоса нежели начнет.
3. Для каждого числа класстера можно брать разный датасет.
4. Перемешивать нужно для того "говорящие" не шли одним за другим.

In [5]:
tracklist = glob(glob_paths['tts'])
# tracklist = tracklist [:int(len(tracklist)*total_amount)]
tracklist = tracklist [:total_amount] 

In [6]:
random.seed(seed)
np.random.seed(seed)
random.shuffle(tracklist)

In [7]:
os.makedirs(f"{paths['root']}/mixed_{k}", exist_ok=True)

Смешивание файлов

In [9]:
# Этот метод не нравится т.к. хочется что бы дорожка была как можно длинее.
# То есть аудио с одинаковой длины объединялись с одинаковыми аудио.

for i in tqdm(range(0, len(tracklist), k)):
    tracks = tracklist[i:i+k]
    tracks = sorted(tracks)
    overlay, min_length = overlap_using_min(tracks)
    audio_names = "_".join([get_file_name(t) for t in tracks])
    audio_name = f"{paths['root']}/mixed_{k}/{audio_names}.flac"
    sf.write(f"{audio_name}", overlay, samplerate=16000)
    df = pd.DataFrame({'mixed_audio': [audio_name],
                       'common_len_idx': [min_length],
                       **{f'audio_{j+1}': [t.replace('\\', '/')] for j, t in enumerate(tracks)}})
    df.to_csv(f"{audio_name.replace('.flac', '.csv')}", index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [11]:
# Get full dataset DF
annotations = glob(f"F:/ISSAI_KSC2_unpacked/mixed_{k}/*.csv")
all_dfs = []
for csv in tqdm(annotations):
    df = pd.read_csv(csv)
    all_dfs.append(df)

concated_df = pd.concat(all_dfs).reset_index(drop=True)
os.makedirs(f"F:/ISSAI_KSC2_unpacked/DIHARD_DATA_INFO", exist_ok=True)
concated_df.to_csv(f"F:/ISSAI_KSC2_unpacked/DIHARD_DATA_INFO/CONCATED_DFS_tts=2000_k_{k}.csv", index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
print(f"Mixed num files: {len(concated_df)} for k = {k} clusters, total_amount_files: {len(tracklist)}")

Mixed num files: 1000 for k = 2 clusters, total_amount_files: 2000


#### To get need to data-format for training.

In [ ]:
# папка где хранять исходники для дириазации - они обрезанные без начало 'F:/ISSAI_KSC2_unpacked/trimmed',
# mixed_2 где хранять смешанные аудиодорожки для двух класстеров
# осторожно - нету папки где храняться targetы - вот эти и займемся

In [ ]:
# 1. Создаем все targetы
# 2. generate `.scp` file

In [ ]:
# Есть csv файл нужно 
# Я хочу сохранить audio_1, audio_2 в отдельные папки 
# Папки будет две - одна tr - train, другая cv - validation
# В каждой внутри папки есть mix и 

In [ ]:
import os.path as ospth

def get_file_name(file_path: str):
    return ospth.splitext(ospth.basename(file_path))[0]

In [ ]:
# 1.

k = 2
csv_path = f"F:/ISSAI_KSC2_unpacked/DIHARD_DATA_INFO/CONCATED_DFS_k_{k}.csv"
full_df = pd.read_csv(csv_path)

for _, row in full_df.iterrows(): 
    # 1. копируем название столбцов
    f = pd.DataFrame(columns=['mixed_audio', 'common_len'])
    mixed_audio = row.iloc[0]
    common_len = row.iloc[1]

    df = df.append({'mixed_audio': mixed_audio, 'common_len': common_len}, ignore_index=True)
    # добавил в df в столбцы 'mixed_audio': 'mixed_audio', 'common_len': 'common_len' 
    i = 1
    for column in full_df.columns[2:]:
        raw_pth_name = row[1]
        audio, sr = librosa.load(raw_pth_name, sr=sr)
        folder_name = f"{paths['root']}/samples_{k}_{i}"
        new_path_file_name = f"{folder_name}/{get_file_name(raw_pth_name)}.flac"
        sf.write(f"{new_path_file_name}", audio[:common_len], samplerate=16000)
        i += 1
        break
    df.to_csv(mixed_audio.replace('flac', 'csv'))
    break

In [ ]:
# 2.

import os

# Пути и файлы для записи
datasets = {
    "train": {
        "mix": "/home/likai/likai/Dataset/wsj0-mix/2speakers/wav8k/min/tr/mix",
        "s1": "/home/likai/likai/Dataset/wsj0-mix/2speakers/wav8k/min/tr/s1",
        "s2": "/home/likai/likai/Dataset/wsj0-mix/2speakers/wav8k/min/tr/s2",
        "scp": {
            "mix": "tr_mix.scp",
            "s1": "tr_s1.scp",
            "s2": "tr_s2.scp",
        },
    },
    "cv": {
        "mix": "/home/likai/likai/Dataset/wsj0-mix/2speakers/wav8k/min/cv/mix",
        "s1": "/home/likai/likai/Dataset/wsj0-mix/2speakers/wav8k/min/cv/s1",
        "s2": "/home/likai/likai/Dataset/wsj0-mix/2speakers/wav8k/min/cv/s2",
        "scp": {
            "mix": "cv_mix.scp",
            "s1": "cv_s1.scp",
            "s2": "cv_s2.scp",
        },
    },
}

# Функция для записи данных в SCP-файл
def write_scp(input_dir, output_file):
    with open(output_file, "w") as f:
        for root, _, files in os.walk(input_dir):
            files.sort()
            for file in files:
                f.write(f"{file} {os.path.join(root, file)}\n")

# Генерация SCP-файлов для всех наборов данных
for split, paths in datasets.items():
    for key, input_dir in paths.items():
        if key == "scp":  # Пропустить словарь с именами файлов
            continue
        write_scp(input_dir, paths["scp"][key])